  ## Normalized NYC --> Region (Final Map)

#### NB setup

In [1]:
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px
import os

In [2]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# set directory explicitly
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [3]:
# Readubg in GeoJSON, convert to geopandas
subplaces = r'data/shapefiles/nyc_subplace_centroid.geojson'
subplaces = gpd.read_file(subplaces)
subplaces.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,900946940.0,-234.0,POINT (-73.12283 41.52707)
1,2,09,009,47535,00213462,0900947535,Milford,Milford town,43,T5,G4040,720,71950,None,C,57442071.0,10218658.0,+41.2250861,-073.0611101,None,None,None,None,0.481270,0.007266,67660729.0,26.12381,900947535.0,-1743.0,POINT (-73.06185 41.22575)
2,3,09,009,58300,00213486,0900958300,Oxford,Oxford town,43,T1,G4040,720,71950,None,A,84803121.0,1531057.0,+41.4440006,-073.1479992,None,None,None,None,0.439399,0.009300,86334178.0,33.33363,900958300.0,248.0,POINT (-73.13503 41.43129)
3,4,09,009,00000,00000000,0900900000,County subdivisions not defined,County subdivisions not defined,00,Z9,G4040,None,None,None,F,0.0,599104136.0,+41.1874659,-072.8153339,None,None,None,None,1.700000,0.064300,599104136.0,231.31411,900900000.0,NaN,POINT (-72.79470 41.25792)
4,5,09,009,44560,00213454,0900944560,Madison,Madison town,43,T1,G4040,720,75700,None,A,93622105.0,1251401.0,+41.3444810,-072.6245213,None,None,None,None,0.739953,0.010206,94873506.0,36.63066,900944560.0,-218.0,POINT (-72.62809 41.34013)


#### Read in each borough, set up function

In [4]:
bk_df = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\bk_subpl_01-09-20.csv')
bx_df = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\bx_subpl_01-09-20.csv')
qn_df = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\qn_subpl_01-09-20.csv')
si_df = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\si_subpl_01-09-20.csv')
mn_df = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\mn_subpl_01-09-20.csv')

decimals = 0 # to round to whole num in function 'clean counties'
# name boroughs to save intermediary files.
brooklyn = 'brooklyn'
bronx = 'bronx'
queens = 'queens'
staten = 'staten'
manhattan = 'manhattan'
def clean_counties(df, county):
    # alter file, convert geoid field to string
    df = df.rename(columns={'date_y-m-d':'id'})
    # remove the 'U' in IDs. This 'U' was placed so pandas will auto read the column as string
    df['id'] = df['id'].str[1:]
    list_date = df.columns.to_list()
    list_date = list_date[1:]  
    # melt based off dates for each subplace
    df = pd.melt(df, id_vars=['id'],value_vars = list_date)
    # convert to dates
    df['variable'] = pd.to_datetime(df['variable'])
    # join to counties
    df = pd.merge(subplaces, df, left_on='GEOID', right_on='id', how = "inner")
    df['date'] = pd.to_datetime(df['variable'])
    df['date'] = df["date"].dt.strftime('%m/%d/%Y')
    df['datetime'] = df['date'].astype(str) + ' 0:00'
    df['value'] = df['value'].astype(float)
    df['value'] = df['value'].apply(lambda x: round(x, decimals)) # round to nearest whole num
    df['value'] = df['value'].astype(int) # remove decimal
    # selecting rows based on value being greater than 0
    df = df.loc[df['value'] >= 1] 
    #renaming columns for hover tool clarity
    df = df.rename(columns = {'NAME':'Name','date':'Date','value':'Trips'})
    df.to_csv(f'data/borough_x_subplace/trips_pop_norm/pop_{county}_norms.csv', index = False) # saving the file
    return(df.head())

In [5]:
# run the function on each df, and borough func(arg1, arg2)
clean_counties(bk_df, brooklyn)
clean_counties(bx_df, bronx)
clean_counties(qn_df, queens)
clean_counties(si_df, staten)
clean_counties(mn_df, manhattan)

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,Name,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry,id,variable,Trips,Date,datetime
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-01,25,01/01/2020,01/01/2020 0:00
1,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-02,26,01/02/2020,01/02/2020 0:00
2,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-03,26,01/03/2020,01/03/2020 0:00
3,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-04,26,01/04/2020,01/04/2020 0:00
4,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-05,26,01/05/2020,01/05/2020 0:00


In [6]:
# csv read all boroughs -- may not be necessary if you create objects from functions
dfbk = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\pop_brooklyn_norms.csv')
dfbk = dfbk.sort_values(by ='Date') # sort here
dfbk['Borough'] = 'Brooklyn'# add column for viz

dfbx = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\pop_bronx_norms.csv')
dfbx = dfbx.sort_values(by ='Date') # sort here
dfbx['Borough'] = 'Bronx'# add column for viz

dfmn = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\pop_manhattan_norms.csv')
dfmn = dfmn.sort_values(by ='Date') # sort here
dfmn['Borough'] = 'Manhattan' # add column for viz

dfqn = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\pop_queens_norms.csv')
dfqn = dfqn.sort_values(by ='Date') # sort here
dfqn['Borough'] = 'Queens' # add column for viz

dfsi = pd.read_csv(r'data\borough_x_subplace\trips_pop_norm\pop_staten_norms.csv')
dfsi = dfsi.sort_values(by ='Date') # sort here
dfsi['Borough'] = 'Staten Island' # add column for viz
# add 'em up
df = pd.concat([dfbk, dfbx, dfmn, dfqn, dfsi], ignore_index =True)

C:\Users\steve\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Plotly Visualization

In [7]:
fig = px.scatter_mapbox(df,# geojson = subplaces, this geojson is not needed if you have lat longs as separate columns in df.
                     lat = 'INTPTLAT', lon = 'INTPTLON',
                     size = 'Trips', # column to reference bubble sizes
                     color ='Borough', # change color based on borough
                     color_discrete_sequence = ['#CCCCFF','#1D819A','#63BB69','#FFE066','#DE7A3F'], # calling in explicit borough colors we created
                     animation_frame = 'Date',
                     animation_group = 'Borough',
                     hover_data = {'Name':True,'Date':True,'Trips':True, 'INTPTLAT':False, 'INTPTLON':False, 'Borough':False}, # specifying hover data layers 
                     #font = dict(family = 'Arial', color = 'white'),
                     size_max = 60) # max bubble size
fig.update_layout(mapbox_zoom = 6.8,
                 mapbox_center = {'lat':40.82, 'lon':-74.25},
                 mapbox_style="carto-darkmatter",
                # size of the viz
                 width = 950,
                 height = 780, 
                 font = dict(family = 'Arial', color = 'black'),
                 title_font_color = 'white',
                # set legend params
                 legend=dict(
                      title = 'Residence',
                      title_font_color = 'white',
                      bgcolor = 'black',
                      bordercolor = 'gray', borderwidth = 1,
                      font = dict(family = 'Arial', color = 'white'),
                      yanchor="top",
                      y=0.33, # location
                      xanchor="left",
                      x=0.02))# location
## THIS IS TO BE ADDED IF STANDALONE VIZ, we are placing in HTML, this is not necessary

# TITLE
# fig.add_annotation(text = "Where NYC Residents Traveled in the<br>Surrounding Metropolitan Region",
#                   align = 'left', x = 0.01, y = 0.95, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 19))

# SUBTITLE
# fig.add_annotation(text = 'January 1, 2020 to September 30, 2020',
#                   align = 'left', x = 0.01, y = 0.86, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 13))

# EXPLANATORY TEXT
fig.add_annotation(text = 'Bubble size = # of Trips',
                  align = 'right', x = 0.02, y = 0.01, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
fig.add_annotation(text = 'Click on/off boroughs in legend to compare or isolate migratory patterns',
                  align = 'right', x = 0.02, y = 0.082, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))

# speed this viz up so it doesn't look like crap..
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 80 
# save the HTML, send it on over
fig.write_html(r'Maps/html_maps/nyc_to_subplace.html', auto_open = True, auto_play = False, include_plotlyjs = 'cdn')